# Part #1 Image Filtering

## 1(1) Image filtering by Cross-Correation
kernel
- r,c 두 방향 다 고려해서 작성
- odd size라고 가정
image padding: 가장 가까운 pixel 값


In [304]:
import cv2
import numpy as np

LENNA_IMG_PATH="lenna.png"
SHAPE_IMG_PATH="shapes.png"

In [305]:
def img_show(img,title=""):
    cv2.imshow(title,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [306]:
def get_padding_size(n,k):
    p_h= (k[0]-1*(n[0]//2==0))//2
    p_w=(k[1]-1*(n[1]//2==0))//2
    return p_h,p_w

In [307]:
# def image_padding_1d(img,kernel):
#     k_size=kernel.shape[0]//2
#     padding=np.concatenate([np.full(k_size, img[0]),img,np.full(k_size,img[-1])])
#     return padding

In [308]:
def image_padding_1d(img,kernel,row=True):
    p_h,p_w=get_padding_size(img.shape,kernel.shape)
    if row==True:
        rowFirst=np.tile(img[:,:1],(1,p_w))
        rowLast=np.tile(img[:,-1:] ,(1,p_w))
        padding=np.concatenate([rowFirst,img,rowLast],1)
    else:
        columnFirst=np.tile(img[:1,:],(p_h,1))
        columnLast=np.tile(img[-1:,:],(p_h,1))
        padding=np.concatenate([columnFirst,img,columnLast],0) 
    return padding

In [309]:
def image_padding_2d(img,kernel):
    #column padding
    padding=image_padding_1d(img,kernel,False)
    #row padding
    padding=image_padding_1d(padding,kernel,True)
    return padding

In [310]:
# def cross_correlation_1d_row(img,kernel):
#     flatImg=img.reshape(-1)
#     kernel=kernel.reshape(-1)
#     paddingImg=image_padding_1d(flatImg,kernel)
#     windows=np.lib.stride_tricks.sliding_window_view(paddingImg,(kernel.shape))
#     filteredImg=np.einsum('i,ji->j',kernel,windows)
#     return filteredImg.reshape(img.shape)

# def cross_correlation_1d_column(img,kernel):
#     trans=img.T
#     flatImg=img.T.reshape(-1)
#     kernel=kernel.reshape(-1)
#     paddingImg=image_padding_1d(flatImg,kernel)
#     windows=np.lib.stride_tricks.sliding_window_view(paddingImg,(kernel.shape))
#     filteredImg=np.einsum('i,ji->j',kernel,windows)
#     return filteredImg.reshape(trans.shape).T

In [311]:
def cross_correlation_1d_row(img,kernel):
    paddingImg=image_padding_1d(img,kernel)
    windows=np.lib.stride_tricks.sliding_window_view(paddingImg,(kernel.shape))
    return np.einsum('ij,klij->kl',kernel,windows)

def cross_correlation_1d_column(img,kernel):
    paddingImg=image_padding_1d(img,kernel,False)
    windows=np.lib.stride_tricks.sliding_window_view(paddingImg,(kernel.shape))
    return np.einsum('ij,klij->kl',kernel,windows)

In [312]:
def cross_correlation_2d(img,kernel):
    paddingImg=image_padding_2d(img,kernel)
    windows=np.lib.stride_tricks.sliding_window_view(paddingImg,(kernel.shape))
    return np.einsum('ij,klij->kl',kernel,windows)

In [313]:
def cross_correlation_1d(img,kernel):
    if kernel.shape[0]==1:
        return cross_correlation_1d_row(img,kernel)
    else:
        return cross_correlation_1d_column(img,kernel)

## 1(2) Gaussian Filter

In [314]:
def get_gaussian_filter_1d(size,sigma):
    size//=2
    x=np.arange(-size,size+1)
    kernel=np.exp(-1*x**2/(2*sigma**2))
    kernel/=kernel.sum()
    return kernel

In [315]:
get_gaussian_filter_1d(5,1)


array([0.05448868, 0.24420134, 0.40261995, 0.24420134, 0.05448868])

In [316]:
def get_gaussian_filter_2d(size,sigma):
    original=get_gaussian_filter_1d(size,sigma)
    return np.einsum('i,j->ji', original,original )

In [317]:
get_gaussian_filter_2d(5,1)

array([[0.00296902, 0.01330621, 0.02193823, 0.01330621, 0.00296902],
       [0.01330621, 0.0596343 , 0.09832033, 0.0596343 , 0.01330621],
       [0.02193823, 0.09832033, 0.16210282, 0.09832033, 0.02193823],
       [0.01330621, 0.0596343 , 0.09832033, 0.0596343 , 0.01330621],
       [0.00296902, 0.01330621, 0.02193823, 0.01330621, 0.00296902]])

In [318]:
def get_gaussian_img(img,size,sigma,dimension=2):
    if dimension==1:
        kernel=get_gaussian_filter_1d(size,sigma)
        column_kernel=np.expand_dims(kernel,axis=1)
        row_kernel=np.expand_dims(kernel,axis=0)
        img=cross_correlation_1d(img,row_kernel)
        img=cross_correlation_1d(img,column_kernel)
        return img
    else:
        return cross_correlation_2d(img,get_gaussian_filter_2d(size,sigma))
    

In [319]:
import os

def getCollage(title,img_path,sizeList=[5,11,17],sigmaList=[1,6,11],dimension=2):
    result=[]
    for i,size in enumerate(sizeList):
        tmp=[]
        for j,sigma in enumerate(sigmaList):
            img=cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img=get_gaussian_img(img,size,sigma,dimension).copy()
            cv2.putText(img,f"{size}X{size} s={sigma}",(10,40),cv2.FONT_HERSHEY_PLAIN,2,(0,0,0),2)
            if j==0:
                tmp=img
            else:
                tmp=cv2.hconcat([tmp,img])
        if i==0:
            result=tmp
        else:
            result=cv2.vconcat([result,tmp])
    directory="./result"
    if not os.path.exists(directory):
        os.makedirs(directory)
    cv2.imwrite(f'./result/part_1_gaussian_filtered_{title}.png',np.array(result))

In [320]:
getCollage("lenna",LENNA_IMG_PATH)
getCollage("shapes",SHAPE_IMG_PATH)

In [321]:
a=get_gaussian_filter_1d(17,11)
c=np.expand_dims(a,axis=1)
print(a.shape)
b=np.expand_dims(a,axis=0)
print(b.shape,len(b))
print(c.shape,len(c))
print(b.reshape(-1))
print(np.flip(b))

(17,)
(1, 17) 1
(17, 1) 17
[0.04967151 0.05284774 0.0557643  0.05835753 0.06056872 0.06234629
 0.06364784 0.06444177 0.06470861 0.06444177 0.06364784 0.06234629
 0.06056872 0.05835753 0.0557643  0.05284774 0.04967151]
[[0.04967151 0.05284774 0.0557643  0.05835753 0.06056872 0.06234629
  0.06364784 0.06444177 0.06470861 0.06444177 0.06364784 0.06234629
  0.06056872 0.05835753 0.0557643  0.05284774 0.04967151]]


In [325]:
img=cv2.imread(LENNA_IMG_PATH, cv2.IMREAD_GRAYSCALE)

img_1d=get_gaussian_img(img,17,11,1)
img_2d=get_gaussian_img(img,17,11)
difference=img_1d-img_2d
np.sum(np.abs(difference))
img_show(difference)